In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import numpy as np

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
learning_rate = 0.001

In [3]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True, 
                                             download=False,
                                             transform=transforms.Compose([
                                                 transforms.Pad(4),         #图像长宽周围都填充4个单位长度（像素）,针对PIL图像显示的时候
                                                 transforms.RandomHorizontalFlip(),  #对PIL.image水平翻转，默认反转概率0.5
                                                 transforms.RandomCrop(32),         #随机切割，切割尺寸为32*32
                                                 transforms.ToTensor()]),
                                             )

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

print(train_dataset)
print(train_dataset[0][0].shape)            #第1个数据的X,这里目前由于是ToTensor的结果，是tensor，暂时没有被padding
print(train_dataset[0][1])            #第1个数据的y
print(train_dataset.targets[0])       #等同于第1个数据的y

batch=next(iter(train_loader))
images,labels = batch
print(images.shape)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
torch.Size([3, 32, 32])
6
6
torch.Size([100, 3, 32, 32])


In [10]:
# 3x3 convolution，定义3*3的卷积层（因为kernel_size为3）,步长默认为1，这样的话kernel_size为3，步长为1，最后图像的长宽不变
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

In [8]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)  # (batch,in_c,w/stride,h/stride) -> (batch,out_c,w/stride,h/stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)         # (batch,out_c,w,h) -> (batch,out_c,w,h)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:  # downsample的作用：相当于那个经典的图的x直接连到了输出，想一想残差原理，以及反向传递的时候避开了downsample走在主线上
            residual = self.downsample(x) # 否则输出out + residual
        out += residual
        out = self.relu(out)
        return out
    
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):     #这个block传递的是ResidualBlock类,layers传递的是[2,2,2]
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)         #(batch,3,32,32) -> (batch,16,32,32)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        #注意，下面的每一个make_layer整体上都是一个残差层,
        self.layer1 = self.make_layer(block, 16, layers[0]) #layers[0]为2，则最后有1+2个block,(b,16,32,32)->(b,16,32,32)->(b,16,32,32)->(b,16,32,32)
        self.layer2 = self.make_layer(block, 32, layers[1], 2) #layers[1]为2,stride为2,则最后有1+2个block，(b,16,32,32)->(这里有一个downsample)(b,32,16,16)->(b,32,16,16)->(b,32,16,16)
        self.layer3 = self.make_layer(block, 64, layers[2], 2) #layers[1]为2,stride为2,则最后有1+2个block，(b,32,16,16)->(这里有一个downsample)(b,64,8,8)->(b,64,8,8)
        self.avg_pool = nn.AvgPool2d(8)           # kernel_size 为8,若输入是(c,w,h),则输出是(c,w-7,h-7),则(b,64,8,8) -> (b,64,1,1)
                                                  # 这里最后用avg pool好，因为这里这8个数值元素更多的是平等关系，为了减小方差
        self.fc = nn.Linear(64, num_classes)      # (b,64,1,1)经过view(out.size(0),-1)之后变成了(b,64),这里out.size(0)为b,再经过fc变成(b,10)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)      # 由(b,64,1,1) 变成了 (b,64)
        out = self.fc(out)
        return out

In [40]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)

In [41]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate, 设置lr逐渐变小，最后converge
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate，每20个epoch的时候更新一次lr
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/80], Step [100/500] Loss: 1.7533
Epoch [1/80], Step [200/500] Loss: 1.5658
Epoch [1/80], Step [300/500] Loss: 1.3540
Epoch [1/80], Step [400/500] Loss: 1.3259
Epoch [1/80], Step [500/500] Loss: 1.1043
Epoch [2/80], Step [100/500] Loss: 1.0082
Epoch [2/80], Step [200/500] Loss: 1.2074
Epoch [2/80], Step [300/500] Loss: 0.9864
Epoch [2/80], Step [400/500] Loss: 0.9434
Epoch [2/80], Step [500/500] Loss: 1.1548
Epoch [3/80], Step [100/500] Loss: 0.9224
Epoch [3/80], Step [200/500] Loss: 0.7737
Epoch [3/80], Step [300/500] Loss: 0.9569
Epoch [3/80], Step [400/500] Loss: 0.8572
Epoch [3/80], Step [500/500] Loss: 0.8203
Epoch [4/80], Step [100/500] Loss: 0.8361
Epoch [4/80], Step [200/500] Loss: 0.8022
Epoch [4/80], Step [300/500] Loss: 0.9255
Epoch [4/80], Step [400/500] Loss: 0.6957
Epoch [4/80], Step [500/500] Loss: 0.9103
Epoch [5/80], Step [100/500] Loss: 0.7111
Epoch [5/80], Step [200/500] Loss: 0.5880
Epoch [5/80], Step [300/500] Loss: 0.6195
Epoch [5/80], Step [400/500] Loss:

Epoch [77/80], Step [200/500] Loss: 0.1745
Epoch [77/80], Step [300/500] Loss: 0.1704
Epoch [77/80], Step [400/500] Loss: 0.1342
Epoch [77/80], Step [500/500] Loss: 0.1151
Epoch [78/80], Step [100/500] Loss: 0.1827
Epoch [78/80], Step [200/500] Loss: 0.1482
Epoch [78/80], Step [300/500] Loss: 0.2630
Epoch [78/80], Step [400/500] Loss: 0.1445
Epoch [78/80], Step [500/500] Loss: 0.1523
Epoch [79/80], Step [100/500] Loss: 0.1650
Epoch [79/80], Step [200/500] Loss: 0.2022
Epoch [79/80], Step [300/500] Loss: 0.1594
Epoch [79/80], Step [400/500] Loss: 0.1632
Epoch [79/80], Step [500/500] Loss: 0.1879
Epoch [80/80], Step [100/500] Loss: 0.1979
Epoch [80/80], Step [200/500] Loss: 0.1215
Epoch [80/80], Step [300/500] Loss: 0.1912
Epoch [80/80], Step [400/500] Loss: 0.1391
Epoch [80/80], Step [500/500] Loss: 0.1229


In [ ]:
# Save the model checkpoint，保存的是模型的参数，则保存和读取跟直接保存模型不太一样,但是这个方法成本小
torch.save(model.state_dict(), 'resnet.ckpt')

In [12]:
# Test the model
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
model.load_state_dict(torch.load('resnet.ckpt'))

model.eval()
with torch.no_grad():         # 使用 torch,no_grad()构建不需要track的上下文环境，这个时候再不会跟踪track各个tensor的梯度
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 87.75 %
